In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from utils import create_sequences

2024-01-23 06:09:48.475138: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 06:09:48.475185: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 06:09:48.475208: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 06:09:48.480918: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('./dataset/anomaly_dataset.csv').set_index('timestamp')
max_val = np.max(df.values)

seq_data = create_sequences(df, seq_len=24)
seq_data /= max_val

In [3]:
train = seq_data[:(365 * 24 * 3)]
val = seq_data[(365 * 24 * 3):(365 * 24 * 4)]
test = seq_data[(365 * 24 * 4):]

print(f'train data: {len(train)}')
print(f'validation data: {len(val)}')
print(f'test data: {len(test)}')

train data: 26280
validation data: 8760
test data: 3601


In [4]:
class Encoder(models.Model):

    def __init__(self, seq_length, latent_dim):
        super(Encoder, self).__init__()

        self.h1 = layers.GRU(128, return_sequences=True)  # (seq_len, input_dim) -> (seq_len, 128))
        self.h2 = layers.GRU(latent_dim, return_sequences=False) # (seq_len , 128) -> (latent_dim)
        self.h3 = layers.RepeatVector(seq_length) # (latent_dim) -> (seq_length, latent_dim)

    def call(self, x):
        x = self.h1(x)
        z = self.h2(x)
        z_rep = self.h3(z)

        return z, z_rep


class Decoder(models.Model):

    def __init__(self, input_dim, latent_dim):
        super(Decoder, self).__init__()

        self.h1 = layers.GRU(latent_dim, return_sequences=True) # (seq_length, latent_dim) -> (seq_len, input_dim)
        self.h2 = layers.GRU(128, return_sequences=True) # (seq_len, input_dim) -> (seq_length, 128)
        self.h3 = layers.TimeDistributed(layers.Dense(input_dim)) # (seq_length, 128) -> (seq_length, input_dim)

    def call(self, x):
        x = self.h1(x)
        x = self.h2(x)
        x = self.h3(x)

        return x



class LstmAE(models.Model):

    def __init__(self, seq_length, input_dim, latent_dim):
        super(LstmAE, self).__init__()

        self.encoder = Encoder(seq_length, latent_dim)
        self.decoder = Decoder(input_dim, latent_dim)

    def call(self, x):
        z, z_rep = self.encoder(x)
        decoded = self.decoder(z_rep)

        return decoded

In [5]:
model = LstmAE(24, 1, 64)
model.build(input_shape=(None, train.shape[1], train.shape[2]))
model.summary()

2024-01-23 06:09:51.139566: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-23 06:09:51.163799: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-23 06:09:51.163840: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-23 06:09:51.166289: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-23 06:09:51.166326: I tensorflow/compile

Model: "lstm_ae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  87552     
                                                                 
 decoder (Decoder)           multiple                  99585     
                                                                 
Total params: 187137 (731.00 KB)
Trainable params: 187137 (731.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
optim = optimizers.Adam(learning_rate=.001)
model.compile(optimizer=optim, loss='mse')

In [7]:
history = model.fit(train, train, epochs=50, batch_size=64, validation_data=(val, val))

Epoch 1/50


2024-01-23 06:10:02.645995: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-01-23 06:10:03.056535: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9fc828e0b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-23 06:10:03.056567: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-01-23 06:10:03.060750: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-23 06:10:03.134322: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


411/411 [==============================] - 12s 16ms/step - loss: 0.0121 - val_loss: 0.0043
Epoch 2/50
411/411 [==============================] - 5s 12ms/step - loss: 0.0030 - val_loss: 0.0024
Epoch 3/50
411/411 [==============================] - 5s 12ms/step - loss: 0.0021 - val_loss: 0.0020
Epoch 4/50
411/411 [==============================] - 5s 12ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 5/50
411/411 [==============================] - 5s 12ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 6/50
411/411 [==============================] - 5s 12ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 7/50
411/411 [==============================] - 5s 11ms/step - loss: 9.6587e-04 - val_loss: 0.0010
Epoch 8/50
411/411 [==============================] - 5s 12ms/step - loss: 8.1670e-04 - val_loss: 9.2612e-04
Epoch 9/50
411/411 [==============================] - 5s 12ms/step - loss: 7.1341e-04 - val_loss: 6.4596e-04
Epoch 10/50
411/411 [==============================] - 5s 11ms/step - loss: 6.253

In [8]:
model.save('./models/new_GRUAE/new_GRUAE')

with open('./models/new_GRUAE/new_GRUAE_hist.pkl', 'wb') as f:
    pickle.dump(history.history, f)

INFO:tensorflow:Assets written to: ./models/new_GRUAE/new_GRUAE/assets


INFO:tensorflow:Assets written to: ./models/new_GRUAE/new_GRUAE/assets
